# Baselines for Sister Projects on special:search 

[T311165](https://phabricator.wikimedia.org/T311165)

In [1]:
import datetime as dt

import pandas as pd
import numpy as np

from wmfdata import spark, mariadb

You are using wmfdata v1.3.1, but v1.3.3 is available.

To update, run `pip install --upgrade git+https://github.com/wikimedia/wmfdata-python.git@release --ignore-installed`.

To see the changes, refer to https://github.com/wikimedia/wmfdata-python/blob/release/CHANGELOG.md


In [2]:
## We're operating with different format for timestamps, so we'll have to be able
## to parse them both with and without milliseconds.

def parse_dt(ts):
    try:
        return(dt.datetime.strptime(ts, '%Y-%m-%dT%H:%M:%S.%fZ'))
    except ValueError: ## no microseconds
        return(dt.datetime.strptime(ts, '%Y-%m-%dT%H:%M:%SZ'))


In [16]:
today = dt.datetime.now(dt.timezone.utc).date()
yesterday = today - dt.timedelta(days = 1)
last_week = today - dt.timedelta(days = 7)

In [13]:
wikis = "'idwiki', 'ptwiki', 'ruwiki'"

## Timestamp Functions

In [18]:
def make_partition_statement(start_ts, end_ts, prefix = ''):
    '''
    This takes the two timestamps and creates a statement that selects
    partitions based on `year`, `month`, and `day` in order to make our
    data gathering not use excessive amounts of data. It assumes that
    `start_ts` and `end_ts` are not more than a month apart, which should
    be a reasonable expectation for this notebook.
    
    An optional prefix can be set to enable selecting partitions for
    multiple tables with different aliases.
    
    :param start_ts: start timestamp
    :type start_ts: datetime.datetime
    
    :param end_ts: end timestamp
    :type end_ts: datetime.datetime
    
    :param prefix: prefix to use in front of partition clauses, "." is added automatically
    :type prefix: str
    '''
    
    if prefix:
        prefix = f'{prefix}.' # adds "." after the prefix
    
    # there are three cases:
    # 1: month and year are the same, output a "BETWEEN" statement with the days
    # 2: months differ, but the years are the same.
    # 3: years differ too.
    # Case #2 and #3 can be combined, because it doesn't really matter
    # if the years are the same in the month-selection or not.
    
    if start_ts.year == end_ts.year and start_ts.month == end_ts.month:
        return(f'''{prefix}year = {start_ts.year}
AND {prefix}month = {start_ts.month}
AND {prefix}day BETWEEN {start_ts.day} AND {end_ts.day}''')
    else:
        return(f'''
(
    ({prefix}year = {start_ts.year}
     AND {prefix}month = {start_ts.month}
     AND {prefix}day >= {start_ts.day})
 OR ({prefix}year = {end_ts.year}
     AND {prefix}month = {end_ts.month}
     AND {prefix}day <= {end_ts.day})
)''')


## Click-Through Rate for Sister Projects

We define the click through rate the proportion of search sessions where the user clicked on one of the results displayed

In [22]:
ctr_query = """

WITH 
ft AS (
    SELECT TO_DATE(coalesce(meta.dt, client_dt, dt)) AS log_date, 
           wiki, 
           COUNT(DISTINCT event.searchsessionid, event.pageviewid) AS n_ft
    FROM event.searchsatisfaction ess
    WHERE {ess_partition_statement}
      AND wiki in ({wiki_db})
      AND useragent.is_bot = false
      AND event.action = "searchResultPage"
      AND event.source = "fulltext"
      AND event.subTest IS NULL
      AND event.isforced IS NULL -- only include non-test users
    GROUP BY TO_DATE(coalesce(meta.dt, client_dt, dt)), wiki
), 
ss AS (
    SELECT TO_DATE(coalesce(meta.dt, client_dt, dt)) AS log_date, 
           wiki, 
           COUNT(DISTINCT event.searchsessionid, event.pageviewid) AS n_ss
    FROM event.searchsatisfaction ess
    WHERE {ess_partition_statement}
      AND wiki in ({wiki_db})
      AND useragent.is_bot = false
      AND event.action = "ssclick"
      AND event.source = "fulltext"
      AND event.subTest IS NULL
      AND event.isforced IS NULL -- only include non-test users
    GROUP BY TO_DATE(coalesce(meta.dt, client_dt, dt)), wiki
)

SELECT ft.log_date,
      ft.wiki, 
      n_ft,
      n_ss
FROM ft JOIN ss ON (ft.log_date = ss.log_date AND ft.wiki = ss.wiki)
"""

In [23]:
search_count = spark.run(ctr_query.format(
            ess_partition_statement = make_partition_statement(last_week, today, prefix = 'ess'),
            wiki_db = wikis))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [25]:
search_count = search_count.loc[(search_count['log_date'] >= last_week) &  
                                (search_count['log_date'] <= yesterday)]

In [29]:
ss_ctr = search_count.groupby(['wiki']).sum().reset_index()
ss_ctr['ctr'] = (ss_ctr['n_ss'] / ss_ctr['n_ft']) * 100
ss_ctr

,wiki,n_ft,n_ss,ctr
0,idwiki,29470,173,0.587038
1,ptwiki,93483,695,0.743451
2,ruwiki,187146,1776,0.948992
